In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
import pandas
import numpy
from matplotlib import pyplot as plt
import seaborn
%matplotlib notebook
import datetime
import general
import discretionary_aua
import vantage_aua
import data_accessing
import combined
import revenue
import costs
import consolidated
import discf
from ipywidgets import widgets
from IPython.display import display
import stats
from IPython.display import clear_output
pandas.set_option('display.float_format', '{:20,.6f}'.format)   #suppressing scientific notation
idx = pandas.IndexSlice

In [3]:
print('Pulling data...')
fund_dic = data_accessing.read_data(data_accessing.fund_data_name,data_accessing.fund_data_types)
nnb_dic = data_accessing.read_data(data_accessing.nnb_data_name,data_accessing.nnb_data_sheet)
discretionary_aua.append_share_class_units(fund_dic) # appending share class units
index_hl_dic = data_accessing.read_data(data_accessing.index_hl_data_name, data_accessing.index_hl_data_sheet)
data_dic = {**fund_dic, **index_hl_dic, **nnb_dic}

input_dic = data_accessing.read_data(data_accessing.assumptions_name, data_accessing.assumptions_sheet)

print('Data pulled successfully.')

Pulling data...
Data pulled successfully.


In [4]:
assumption_button = widgets.Button(description='Update assumptions',layout=widgets.Layout(width='20%',height='40px'))
assumption_button.style.button_color = 'lightgreen'
def update_assumption(b):
    global input_dic
    input_dic = data_accessing.read_data(data_accessing.assumptions_name, data_accessing.assumptions_sheet)
    print ('Assumptions updated successfully...')

assumption_button.on_click(update_assumption)
display(assumption_button)

Button(description='Update assumptions', layout=Layout(height='40px', width='20%'), style=ButtonStyle(button_color='lightgreen'))

In [5]:
revenue_types = ['Semi-annual revenue','Annual revenue - FY','Annual revenue - CY','Consolidated semi-annual revenue','Consolidated annual revenue - FY','Consolidated annual revenue - CY']
box_layout = widgets.Layout(display='flex', flex_flow='column', align_items='stretch',border='solid',width='30%')
items = {}
for w in revenue_types:
    items[w] = widgets.Button(description=w,layout=widgets.Layout(width='100%', height='40px'))
    items[w].style.button_color = 'lightgreen'
left_box = widgets.VBox([list(items.values())[0],list(items.values())[1],list(items.values())[2]],layout=box_layout)
right_box = widgets.VBox([list(items.values())[3],list(items.values())[4],list(items.values())[5]],layout=box_layout)
items['Semi-annual revenue'].on_click(lambda x: display(revenue.semi_revenue(data_dic,input_dic)))
items['Annual revenue - FY'].on_click(lambda x: display(revenue.annual_revenue(data_dic,input_dic)))
items['Annual revenue - CY'].on_click(lambda x: display(revenue.annual_revenue(data_dic,input_dic,cal_year=True)))
items['Consolidated semi-annual revenue'].on_click(lambda x: display(consolidated.revenue_analysis(data_dic,input_dic)))
items['Consolidated annual revenue - FY'].on_click(lambda x: display(consolidated.annual_revenue_analysis(data_dic,input_dic)))
items['Consolidated annual revenue - CY'].on_click(lambda x: display(consolidated.annual_revenue_analysis(data_dic,input_dic,cal_year=True)))
widgets.HBox([left_box,right_box])



HBox(children=(VBox(children=(Button(description='Semi-annual revenue', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual revenue - FY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual revenue - CY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%')), VBox(children=(Button(description='Consolidated semi-annual revenue', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Consolidated annual revenue - FY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Consolidated annual revenue - CY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%'))))

,renewal_income,management_fee,stockbroking_commission,stockbroking_income,interest_on_cash,hlf_amc,platform_fee,pms_advice,advice_fee,funds_library,paper_income,other_income,currency_revenue,interest_on_reserve,cash_service
financial_year,,,,,,,,,,,,,,,
2018,"5,394,600.000000","19,744,060.000000","71,098,800.000000","2,626,000.000000","43,374,225.536675","67,325,161.479452","198,336,043.061819","13,098,998.803247","11,400,000.000000","8,040,000.000000","5,029,985.600000","3,784,000.000000","2,211,000.000000","1,412,180.000000",0.000000
2019,"5,373,043.178400","20,328,801.857925","78,836,493.767892","2,732,090.400000","78,841,282.964364","74,688,878.569380","232,589,664.939774","15,051,308.885179","11,400,000.000000","8,201,604.000000","5,781,516.000000","3,784,000.000000","2,255,441.100000","1,461,751.415617","103,125.000000"
2020,"5,351,572.497859","20,930,861.483342","82,779,113.916509","2,842,466.852160","107,719,532.634527","84,482,563.746727","248,782,803.476697","16,149,600.075260","11,400,000.000000","8,366,456.240400","6,614,724.000000","3,784,000.000000","2,300,775.466110","1,513,062.924739","387,500.000000"
2021,"5,330,187.614158","21,550,751.760810","86,918,904.853594","2,957,302.512987","124,669,670.534757","94,021,685.032974","269,726,619.704854","17,288,148.914685","11,400,000.000000","8,534,622.010832","7,568,011.200000","3,784,000.000000","2,347,021.052979","1,566,175.609451","687,500.000000"
2022,"5,308,888.184451","22,189,000.764526","91,265,727.108023","3,076,777.534512","142,864,427.809736","104,372,864.810839","305,961,272.056497","18,508,345.966162","11,400,000.000000","8,706,167.913250","8,658,681.600000","3,784,000.000000","2,394,196.176144","1,621,152.695987","987,500.000000"
2023,"5,292,966.834101","22,846,152.208178","95,829,934.334611","3,201,079.346906","163,006,356.774435","115,351,468.501051","345,970,621.797737","19,816,351.201191","11,400,000.000000","8,881,161.888306","9,896,572.800000","3,784,000.000000","2,442,319.519284","1,678,059.629998","1,287,500.000000"
2024,"5,292,966.834101","23,522,765.908129","100,622,397.975379","3,330,402.952521","183,930,152.070794","127,355,506.539011","387,582,269.754483","21,195,557.231216","11,400,000.000000","9,059,673.242261","11,052,242.400000","3,784,000.000000","2,491,410.141622","1,736,964.154455","1,587,500.000000"
2025,"5,292,966.834101","24,219,418.260313","105,654,533.154143","3,464,951.231803","201,928,247.333683","139,332,517.364404","423,867,709.612905","22,586,176.957373","11,400,000.000000","9,241,772.674430","11,744,352.000000","3,784,000.000000","2,541,487.485468","1,797,936.390298","1,887,500.000000"
2026,"5,292,966.834101","24,936,702.731258","110,938,325.866090","3,604,935.261568","219,836,315.873807","152,401,529.249596","460,141,364.946703","24,040,272.747209","11,400,000.000000","9,427,532.305187","12,378,991.200000","3,784,000.000000","2,592,571.383926","1,861,048.919902","2,187,500.000000"


In [6]:
costs_types = ['Semi-annual costs','Annual costs - FY','Annual costs - CY','Consolidated semi-annual costs','Consolidated annual costs - FY','Consolidated annual costs - CY']
items2 = {}
for w2 in costs_types:
    items2[w2] = widgets.Button(description=w2,layout=widgets.Layout(width='100%', height='40px'))
    items2[w2].style.button_color = 'lightgreen'
left_box2 = widgets.VBox([list(items2.values())[0],list(items2.values())[1],list(items2.values())[2]],layout=box_layout)
right_box2 = widgets.VBox([list(items2.values())[3],list(items2.values())[4],list(items2.values())[5]],layout=box_layout)
items2['Semi-annual costs'].on_click(lambda x: display(costs.semi_costs(input_dic)))
items2['Annual costs - FY'].on_click(lambda x: display(costs.annual_costs(input_dic)))
items2['Annual costs - CY'].on_click(lambda x: display(costs.annual_costs(input_dic,cal_year=True)))
items2['Consolidated semi-annual costs'].on_click(lambda x: display(consolidated.costs_analysis(input_dic)))
items2['Consolidated annual costs - FY'].on_click(lambda x: display(consolidated.costs_analysis(input_dic)))
items2['Consolidated annual costs - CY'].on_click(lambda x: display(consolidated.annual_costs_analysis(input_dic,cal_year=True)))
widgets.HBox([left_box2,right_box2])

HBox(children=(VBox(children=(Button(description='Semi-annual costs', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual costs - FY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual costs - CY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%')), VBox(children=(Button(description='Consolidated semi-annual costs', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Consolidated annual costs - FY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Consolidated annual costs - CY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%'))))

In [7]:
ops = [i for i in list(range(general.recent_end_year, general.recent_end_year+10))]
a_slider = widgets.SelectMultiple(options=ops, description='Years for comparison')
display(a_slider)

compare1 = ['NNB comparison','AUA comparison']
box_layout2 = widgets.Layout(display='flex', flex_flow='row', align_items='stretch',border='solid',width='30%')
items3 = {}
for w3 in compare1:
    items3[w3] = widgets.Button(description=w3,layout=widgets.Layout(width='100%', height='40px'))
    items3[w3].style.button_color = 'lightgreen'
box3 = widgets.Box([list(items3.values())[0],list(items3.values())[1]],layout=box_layout2)
items3['NNB comparison'].on_click(lambda x: display(consolidated.get_nnb_compare(data_dic, input_dic,list(a_slider.value))))
items3['AUA comparison'].on_click(lambda x: display(consolidated.get_aua_compare(data_dic, input_dic,list(a_slider.value))))


display(box3)

SelectMultiple(description='Years for comparison', options=(2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027), value=())

Box(children=(Button(description='NNB comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='AUA comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='30%'))

In [8]:
compare2 = ['Interim revenue comparison','Annual revenue comparison', 'Interim costs comparison', 'Annual costs comparison']
box_layout3 = widgets.Layout(display='flex', flex_flow='row', align_items='stretch',border='solid',width='50%')
items4 = {}
for w4 in compare2:
    items4[w4] = widgets.Button(description=w4,layout=widgets.Layout(width='100%', height='40px'))
    items4[w4].style.button_color = 'lightgreen'
top_box4 = widgets.HBox([list(items4.values())[0],list(items4.values())[1]],layout=box_layout3)
bottom_box4 = widgets.HBox([list(items4.values())[2],list(items4.values())[3]],layout=box_layout3)
items4['Interim revenue comparison'].on_click(lambda x: display(consolidated.get_revenue_compare(data_dic, input_dic,True,list(a_slider.value))))
items4['Annual revenue comparison'].on_click(lambda x: display(consolidated.get_revenue_compare(data_dic, input_dic,False,list(a_slider.value))))
items4['Interim costs comparison'].on_click(lambda x: display(consolidated.get_costs_compare(input_dic,True,list(a_slider.value))))
items4['Annual costs comparison'].on_click(lambda x: display(consolidated.get_costs_compare(input_dic,False,list(a_slider.value))))

widgets.VBox([top_box4, bottom_box4])

VBox(children=(HBox(children=(Button(description='Interim revenue comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual revenue comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%')), HBox(children=(Button(description='Interim costs comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual costs comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%'))))

Actual                       \
financial_year                      2018                 2019   
Platform fees          97,800,000.000000                  nan   
Net renewal income      2,700,000.000000                  nan   
Management fees        15,800,000.000000                  nan   
HL Fund AMC            33,300,000.000000                  nan   
Stockbroking income    34,400,000.000000                  nan   
Interest receivable    18,200,000.000000                  nan   
Adviser charges         5,700,000.000000                  nan   
Funds Library           4,000,000.000000                  nan   
Cash Service                    0.000000                  nan   
Other income            4,300,000.000000                  nan   
Total revenue         216,200,000.000000             0.000000   

                             WP forecast                       
financial_year                      2018                 2019  
Platform fees         198,336,043.061819   232,589,664.939774  
Net renewal income      5,394,600.000000     5,373,043.178400  
Management fees        32,843,058.803247    35,380,110.743104  
HL Fund AMC            67,325,161.479452    74,688,878.569380  
Stockbroking income    73,724,800.000000    81,568,584.167892  
Interest receivable    43,374,225.536675    78,841,282.964364  
Adviser charges        11,400,000.000000    11,400,000.000000  
Funds Library           8,040,000.000000     8,201,604.000000  
Cash Service                    0.000000       103,125.000000  
Other income            8,813,985.600000     9,565,516.000000  
Total revenue         449,251,874.481193   537,711,809.562915

In [9]:
'''=====current values=======
discounted cash flow period: 9 years
perpetuity growth rate: 0.035
cash flow discounted rate: 0.085
'''
dcf_period = widgets.IntText(value=9,placeholder='Integer',description='DCF period - No. of years:')
display(dcf_period)
perp_growth = widgets.FloatText(value=0.035,placeholder='Percentage as decimals',description='Perpetuity growth rate:')
display(perp_growth)
disct_rate = widgets.FloatText(value=0.085,placeholder='Percentage as decimals',description='Cash flow discount rate:')
display(disct_rate)
current_value = widgets.Checkbox(value=False, description='Fair value now')
display(current_value)

dis_cash = ['Discounted cash flow','Fair value']
items5 = {}
for w5 in dis_cash:
    items5[w5] = widgets.Button(description=w5, layout=widgets.Layout(width='100%', height='40px'))
    items5[w5].style.button_color = 'lightgreen'
fair = widgets.HBox([list(items5.values())[0],list(items5.values())[1]],layout=box_layout3)
items5['Discounted cash flow'].on_click(lambda x: display(discf.disc_cash_flow(data_dic, input_dic,now=current_value.value,dcf_p=dcf_period.value,disc_rate=disct_rate.value)))
items5['Fair value'].on_click(lambda x: display(discf.fair_value(data_dic, input_dic,now=current_value.value,dcf_p=dcf_period.value,disc_rate=disct_rate.value,pep_rate=perp_growth.value)))    
display(fair)

IntText(value=9, description='DCF period - No. of years:')

FloatText(value=0.035, description='Perpetuity growth rate:')

FloatText(value=0.085, description='Cash flow discount rate:')

Checkbox(value=False, description='Fair value now')

HBox(children=(Button(description='Discounted cash flow', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Fair value', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%'))

,HL
Terminal value,"14,350,798,417.151114"
Enterprise value,"9,644,951,026.581278"
Net debt&cash,"33,485,311.750000"
Fair value,"9,678,436,338.331278"
No. of shares,474402854
Fair value per share,20.401303


,HL
Terminal value,"13,208,608,253.222986"
Enterprise value,"8,778,394,642.287281"
Net debt&cash,"33,485,311.750000"
Fair value,"8,811,879,954.037281"
No. of shares,474402854
Fair value per share,18.574677


In [10]:
radio1 = widgets.RadioButtons(options=['month_no','quarter_no','half_no','financial_year','calendar_year'],description='Period',disabled=False)
display(radio1)

others = ['HLF composite monthly return','Total NNB hlf Algo','NNB distribution hlf Algo','Total AUA','% Revenue distribution','Avg AUA distribution', 'Total NNB nnc Algo', 'NNB distribution nnc Algo']
items6 = {}
for w6 in others:
    items6[w6] = widgets.Button(description=w6, layout=widgets.Layout(width='100%', height='40px'))
    items6[w6].style.button_color = 'lightgreen'
other_box1 = widgets.VBox([list(items6.values())[0],list(items6.values())[1],list(items6.values())[2],list(items6.values())[3]],layout=box_layout)
other_box2 = widgets.VBox([list(items6.values())[4],list(items6.values())[5],list(items6.values())[6],list(items6.values())[7]],layout=box_layout)
items6['HLF composite monthly return'].on_click(lambda x: display(discretionary_aua.get_composite_return(data_dic)))
items6['Total NNB hlf Algo'].on_click(lambda x: display(combined.total_nnb(data_dic, input_dic, opt=radio1.value)))
items6['NNB distribution hlf Algo'].on_click(lambda x: display(combined.nnb_distribution(data_dic, input_dic, opt=radio1.value)))
items6['Total AUA'].on_click(lambda x: display(combined.total_aua(data_dic, input_dic)))
items6['% Revenue distribution'].on_click(lambda x: display(stats.summary_revenue_dist_percent(data_dic, input_dic)))
items6['Avg AUA distribution'].on_click(lambda x: display(stats.summary_avg_aua_dist(data_dic, input_dic,radio1.value)))
items6['Total NNB nnc Algo'].on_click(lambda x: display(combined.total_nnb_ci_clientAlgo(data_dic, input_dic, opt=radio1.value)))
items6['NNB distribution nnc Algo'].on_click(lambda x: display(combined.total_nnb_distribution_clientAlgo(data_dic, input_dic, opt=radio1.value)))

widgets.HBox([other_box1, other_box2])

#Financial year based

RadioButtons(description='Period', options=('month_no', 'quarter_no', 'half_no', 'financial_year', 'calendar_year'), value='month_no')

HBox(children=(VBox(children=(Button(description='HLF composite monthly return', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Total NNB hlf Algo', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='NNB distribution hlf Algo', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Total AUA', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%')), VBox(children=(Button(description='% Revenue distribution', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Avg AUA distribution', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Total NNB nnc Algo', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='NNB distribution nnc Algo', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%'))))

,,,,,,cash_service_aua,pms_hlf_aua,pms_others_aua,thirdparty_hlf_aua,vantage_cash_aua,vantage_hl_shares_aua,vantage_hlf_aua,vantage_other_funds_aua,vantage_other_shares_aua,hlf_aua,...,total_hlf_aua,total_funds_aua,total_assets_aua,Funds,Shares,HLF,Cash,SIPP,ISA,F&S
month_end,financial_year,quarter_no,half_no,calendar_year,month_no,,,,,,,,,,,,,,,,,,,,,
2017-12-29,2018,2,1,2017,12,0.000000,"3,562,941,176.470588","37,058,823.529412","99,028,884.537726","8,600,000,000.000000",0.000000,"5,248,530,880.499459","40,052,440,234.962814","25,100,000,000.000000","5,347,559,765.037185",...,"8,910,500,941.507772","49,000,000,000.000000","82,700,000,000.000000","49,000,000,000.000000","25,100,000,000.000000","8,947,559,765.037186","8,600,000,000.000000","26,103,000,000.000000","31,640,000,000.000000","21,357,000,000.000000"
2018-01-31,2018,3,2,2018,1,0.000000,"3,545,059,002.367292","38,185,377.138815","99,088,483.479752","8,671,697,915.840517","3,400,000,000.000000","5,263,757,116.264976","40,298,678,588.801376","24,815,573,069.036301","5,362,845,599.744728",...,"8,907,904,602.112020","49,244,768,568.052216","86,132,039,552.929016","49,244,768,568.052216","28,215,573,069.036301","8,946,089,979.250835","8,671,697,915.840517","27,241,102,407.229561","33,019,518,069.369167","22,288,174,696.824188"
2018-02-28,2018,3,2,2018,2,0.000000,"3,459,885,618.226143","38,612,022.174273","97,277,781.134390","8,743,395,831.681034","3,158,384,925.975774","5,179,857,383.798903","39,479,715,233.793251","24,246,923,158.226643","5,277,135,164.933292",...,"8,737,020,783.159435","48,255,348,039.126961","84,404,051,955.010406","48,255,348,039.126961","27,405,308,084.202415","8,775,632,805.333710","8,743,395,831.681034","26,698,832,923.821297","32,362,221,725.843994","21,844,499,664.944698"
2018-03-30,2018,3,2,2018,3,0.000000,"3,362,069,697.593570","40,219,719.278693","95,672,308.950429","8,886,791,663.362068","2,989,071,332.436070","5,118,902,341.725142","38,996,857,978.204369","24,097,749,342.886185","5,214,574,650.675571",...,"8,576,644,348.269141","47,613,722,045.752205","83,587,334,384.436539","47,613,722,045.752205","27,086,820,675.322254","8,616,864,067.547834","8,886,791,663.362068","26,461,064,839.296211","32,074,017,987.025711","21,649,962,141.242355"
2018-04-30,2018,4,2,2018,4,0.000000,"3,525,015,458.740598","43,982,269.802225","101,078,113.620265","8,995,665,278.817482","3,275,531,628.532975","5,424,420,476.587593","41,135,501,362.047462","25,859,613,217.140610","5,525,498,590.207858",...,"9,050,514,048.948456","50,229,997,680.798141","88,360,807,805.289185","50,229,997,680.798141","29,135,144,845.673584","9,094,496,318.750681","8,995,665,278.817482","27,981,297,325.326302","33,916,724,030.698547","22,893,788,720.721519"
2018-05-31,2018,4,2,2018,5,0.000000,"3,580,750,668.950405","45,595,842.697652","103,065,659.551448","9,050,102,086.545189","3,490,605,652.759086","5,539,265,654.620888","41,960,210,810.409721","26,645,750,808.376492","5,642,331,314.172336",...,"9,223,081,983.122742","51,228,888,636.230118","90,415,347,183.910873","51,228,888,636.230118","30,136,356,461.135578","9,268,677,825.820393","9,050,102,086.545189","28,640,370,221.846729","34,715,600,268.905128","23,433,030,181.510963"
2018-06-29,2018,4,2,2018,6,0.000000,"3,571,091,496.492855","46,529,236.278559","103,235,624.880025","9,104,538,894.272896","3,608,667,563.930014","5,557,779,060.716030","42,033,211,646.692329","26,721,338,838.929161","5,661,014,685.596055",...,"9,232,106,182.088909","51,311,847,065.059799","90,746,392,362.191879","51,311,847,065.059799","30,330,006,402.859177","9,278,635,418.367470","9,104,538,894.272896","28,752,494,637.708755","34,851,508,651.768188","23,524,768,339.943527"
2018-07-31,2019,1,1,2018,7,"5,000,000.000000","3,581,172,099.696703","56,609,839.482408","107,746,267.669544","9,634,128,011.592075","3,608,667,563.930014","5,888,559,531.947391","44,153,997,868.837982","28,275,298,969.792789","5,996,305,799.616935",...,"9,577,477,899.313637","53,788,085,607.63

In [11]:
nnb_radio = widgets.RadioButtons(options=['day','month','semi-annual','quarter','annual'],description='Period',disabled=False)
nnb_dropdown = widgets.Dropdown(options=['no_select',None],description='Fund option',disabled=False)
unit_dropdown = widgets.Dropdown(options=['acc','inc'],description='Unit type',disabled=False)
display(nnb_radio)
display(nnb_dropdown)
display(unit_dropdown)

nnb_b = widgets.Button(description='Implied HLF nnb period-to-date', layout=widgets.Layout(width='50%', height='40px'))
nnb_b.style.button_color = 'lightgreen'

nnb_b2 = widgets.Button(description='HLF unit change period-to-date', layout=widgets.Layout(width='50%', height='40px'))
nnb_b2.style.button_color = 'lightgreen'
nnb_b.on_click(lambda x: display(stats.hlf_to_date_implied_nnb(data_dic,typ=nnb_radio.value,fund_opt=nnb_dropdown.value)))
nnb_b2.on_click(lambda x: display(stats.hlf_to_date_unit_change(data_dic, unit_type=unit_dropdown.value,typ=nnb_radio.value,fund_opt=nnb_dropdown.value)))

display(nnb_b)
display(nnb_b2)

RadioButtons(description='Period', options=('day', 'month', 'semi-annual', 'quarter', 'annual'), value='day')

Dropdown(description='Fund option', options=('no_select', None), value='no_select')

Dropdown(description='Unit type', options=('acc', 'inc'), value='acc')

Button(description='Implied HLF nnb period-to-date', layout=Layout(height='40px', width='50%'), style=ButtonStyle(button_color='lightgreen'))

Button(description='HLF unit change period-to-date', layout=Layout(height='40px', width='50%'), style=ButtonStyle(button_color='lightgreen'))

In [12]:
cash_radio = widgets.RadioButtons(options=['monthly','quarter_no','half_no','financial_year','calendar_year'],description='Period',disabled=False)
display(cash_radio)


cash_b = widgets.Button(description='Cash margin', layout=widgets.Layout(width='50%', height='40px'))
cash_b.style.button_color = 'lightgreen'
cash_b.on_click(lambda x: display(stats.cash_margin(data_dic,cash_radio.value)))
display(cash_b)

RadioButtons(description='Period', options=('monthly', 'quarter_no', 'half_no', 'financial_year', 'calendar_year'), value='monthly')

Button(description='Cash margin', layout=Layout(height='40px', width='50%'), style=ButtonStyle(button_color='lightgreen'))

In [13]:
eps_radio = widgets.RadioButtons(options=[True, False],description='Calendar year?',disabled=False)
display(eps_radio)

eps_summary = widgets.Button(description='Net earnings and EPS', layout=widgets.Layout(width='50%', height='40px'))
eps_summary.style.button_color = 'lightgreen'
eps_summary.on_click(lambda x: display(stats.summary_total(data_dic, input_dic, cal=eps_radio.value)))
display(eps_summary)


RadioButtons(description='Calendar year?', options=(True, False), value=True)

Button(description='Net earnings and EPS', layout=Layout(height='40px', width='50%'), style=ButtonStyle(button_color='lightgreen'))

In [14]:
hlf_radio = widgets.RadioButtons(options=['month_no','quarter_no','half_no','financial_year','calendar_year'],description='Period',disabled=False)
display(hlf_radio)


buttom3 = ['HLF revenue margin','Monthly revenue', 'HLF avg fund size','Quarter Trading update revenue']
items7 = {}
for w7 in buttom3:
    items7[w7] = widgets.Button(description=w7, layout=widgets.Layout(width='100%', height='40px'))
    items7[w7].style.button_color = 'lightgreen'

top_m = widgets.HBox([list(items7.values())[0],list(items7.values())[1]],layout=box_layout3)
bottom_m = widgets.HBox([list(items7.values())[2],list(items7.values())[3]],layout=box_layout3)

items7['HLF revenue margin'].on_click(lambda x: display(stats.hlf_revenue_margin(data_dic,input_dic,hlf_radio.value)))
items7['Monthly revenue'].on_click(lambda x: display(revenue.monthly_revenue(data_dic, input_dic)))
items7['HLF avg fund size'].on_click(lambda x: display(stats.avg_hlf_size(data_dic, input_dic,hlf_radio.value)))
items7['Quarter Trading update revenue'].on_click(lambda x: display(stats.quarter_revenue(data_dic, input_dic)))


widgets.VBox([top_m, bottom_m])

RadioButtons(description='Period', options=('month_no', 'quarter_no', 'half_no', 'financial_year', 'calendar_year'), value='month_no')

VBox(children=(HBox(children=(Button(description='HLF revenue margin', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Monthly revenue', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%')), HBox(children=(Button(description='HLF avg fund size', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Quarter Trading update revenue', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%'))))

,,,,,,cash_interest,cash_service,platform_fee,pms_advice,hlf_amc
month_end,financial_year,quarter_no,half_no,calendar_year,month_no,,,,,
2017-12-29,2018,2,1,2017,12,"3,884,545.925609",0.000000,"16,537,500.000000","1,200,000.000000","5,728,755.657534"
2018-01-31,2018,3,2,2018,1,"3,943,732.024770",0.000000,"16,620,109.391718","1,194,414.793169","5,864,802.657534"
2018-02-28,2018,3,2,2018,2,"4,016,834.711942",0.000000,"16,286,179.963205","1,166,165.880133","5,100,278.095890"
2018-03-30,2018,3,2,2018,3,"4,133,938.064642",0.000000,"16,069,631.190441","1,134,096.472291","5,642,198.383562"
2018-04-30,2018,4,2,2018,4,"4,268,727.807984",0.000000,"16,952,624.217269","1,189,665.909514","5,548,002.328767"
2018-05-31,2018,4,2,2018,5,"4,370,059.266524",0.000000,"17,289,749.914728","1,208,782.170549","6,012,392.589041"
2018-06-29,2018,4,2,2018,6,"4,440,933.660814",0.000000,"17,317,748.384458","1,205,873.577590","5,857,487.424658"
2018-07-31,2019,1,1,2018,7,"4,782,146.842875","1,041.666667","18,153,478.892576","1,212,593.979726","6,015,345.021821"
2018-08-31,2019,1,1,2018,8,"4,915,329.731953","2,083.333333","18,270,446.625155","1,213,542.115133","6,038,979.169414"


In [15]:
display(a_slider)

client_options = ['Total No. of clients','Net new clients']
items8 = {}
for w8 in client_options:
    items8[w8] = widgets.Button(description=w8, layout=widgets.Layout(width='100%', height='40px'))
    items8[w8].style.button_color = 'lightgreen'
c_fair = widgets.HBox([list(items8.values())[0],list(items8.values())[1]],layout=box_layout3)
items8['Total No. of clients'].on_click(lambda x: display(combined.total_client_predt(data_dic, input_dic).loc[list(a_slider.value)]))
items8['Net new clients'].on_click(lambda x: display(combined.net_new_client_predt(data_dic, input_dic).loc[list(a_slider.value)]))
display(c_fair)

SelectMultiple(description='Years for comparison', options=(2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027), value=())

HBox(children=(Button(description='Total No. of clients', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Net new clients', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%'))

financial_year  quarter_no
2018            1                   29,000.000000
                2                   32,000.000000
                3                   45,674.000000
                4                   31,820.000000
2019            1                   32,774.000000
                2                   33,758.000000
                3                   52,156.000000
                4                   38,757.000000
Name: No. of clients, dtype: float64

In [16]:
# %load_ext autoreload
# %autoreload 2